In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
import numpy as np

from scipy.stats import unitary_group

In [4]:
from SPTOptimization.gradients import *
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries
from SPTOptimization.utils import (
    tenpy_to_np_transfer_matrix,
    get_left_identity_environment,
    get_right_identity_environment
)

# Load data

In [5]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg/0_50.h5"

In [6]:
with h5py.File(DATA_DIR, 'r') as f:
    data = hdf5_io.load_from_hdf5(f)

In [7]:
psi = data['wavefunction']

# Definitions

In [8]:
unitary_sampler = unitary_group(2)

In [9]:
unitary_sampler.rvs()

array([[-0.10123215+0.03019309j, -0.99307082+0.05148575j],
       [ 0.88579198+0.45189933j, -0.08012604-0.06884322j]])

In [10]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [11]:
test = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z]
)

In [12]:
test.compute_expectation()

array(0.93061628)

In [13]:
test.compute_svd_approximate_expectation()

(0.9306162804445224+0j)

# Tests

In [14]:
test.right_transfer_matrices

[<npc.Array shape=(8, 8, 8, 8) labels=['vL', 'vR', 'vL*', 'vR*']>,
 <npc.Array shape=(8, 8, 8, 8) labels=['vL', 'vR', 'vL*', 'vR*']>]

In [15]:
test.right_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

## identity_environment_functions

In [16]:
ir = get_right_identity_environment(psi, 100)

In [17]:
ir

<npc.Array shape=(8, 8) labels=['vL', 'vL*']>

In [18]:
ir.to_ndarray()

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [19]:
il = get_left_identity_environment(psi, 100)

In [20]:
il

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

In [21]:
il.to_ndarray()

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

## expectation_gradient_from_environments
Just check shape

In [22]:
expectation_gradient_from_environments(psi, 100)

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [23]:
expectation_gradient_from_environments(psi, 100, left_environment=il)

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [24]:
expectation_gradient_from_environments(psi, 100, right_environment=ir)

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [25]:
expectation_gradient_from_environments(psi, 100, left_environment=il, right_environment=ir)

<npc.Array shape=(2, 2) labels=['p*', 'p']>

## expectation gradients

In [26]:
psi.get_B(100)

<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>

In [27]:
a = psi.get_B(100)

In [28]:
a

<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>

In [29]:
a.legs

[LegCharge(ChargeInfo([], []), qconj=+1,
 array([0, 8]), array([], shape=(1, 0), dtype=int64)),
 LegCharge(ChargeInfo([], []), qconj=+1,
 array([0, 2]), array([], shape=(1, 0), dtype=int64)),
 LegCharge(ChargeInfo([], []), qconj=-1,
 array([0, 8]), array([], shape=(1, 0), dtype=int64))]

In [30]:
right_gradients = expectation_gradients(
    psi,
    test.right_transfer_matrices,
    test.right_symmetry_index+1,
    left_environment=test.right_projected_symmetry_state,
    right_environment=None
)

In [31]:
len(test.right_transfer_matrices)

2

In [32]:
len(right_gradients)

2

In [33]:
right_gradients[0]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [34]:
right_gradients[0].to_ndarray()

array([[0.17654904+0.j, 0.68257359+0.j],
       [0.68257359+0.j, 0.17654904+0.j]])

In [35]:
right_gradients[0].shape

(2, 2)

In [36]:
right_gradients[1]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [37]:
right_gradients[1].to_ndarray()

array([[ 6.82573626e-01+0.j,  1.05236871e-08+0.j],
       [ 1.05236870e-08+0.j, -6.82573545e-01+0.j]])

In [38]:
right_gradients[1].shape

(2, 2)

In [39]:
# Careful with np/npc types...
#first_order_correction_from_gradient_one_site(right_gradients[0], np_Z)

In [40]:
gradient = (
    test.left_expectation
    *test.symmetry_transfer_matrix_singular_vals[0]
    *right_gradients[0].to_ndarray()
)
delta_U = np_X

In [41]:
np.sum(gradient*delta_U)

(0.9306162804445222+0j)

In [42]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X + np_X, np_Z]
)

In [43]:
test2.compute_expectation()

array(1.86123256)

In [44]:
test2.expectation - test.expectation

0.9306162804445213

Repeat for second site

In [45]:
right_gradients[1]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [46]:
right_gradients[1].to_ndarray()

array([[ 6.82573626e-01+0.j,  1.05236871e-08+0.j],
       [ 1.05236870e-08+0.j, -6.82573545e-01+0.j]])

In [47]:
right_gradients[1].shape

(2, 2)

In [48]:
# Careful with np/npc types...
#first_order_correction_from_gradient_one_site(right_gradients[0], np_Z)

In [49]:
gradient = (
    test.left_expectation
    *test.symmetry_transfer_matrix_singular_vals[0]
    *right_gradients[1].to_ndarray()
)
delta_U = np_Z

In [50]:
np.sum(gradient*delta_U)

(0.930616280444522+0j)

In [51]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z + np_Z]
)

In [52]:
test2.compute_expectation()

array(1.86123256)

In [53]:
test2.expectation - test.expectation

0.9306162804445213

Repeat on left side

In [54]:
left_gradients = expectation_gradients(
    psi,
    test.left_transfer_matrices[::-1],
    test.left_symmetry_index-1,
    left_environment=None,
    right_environment=test.left_projected_symmetry_state,
    mps_form="A"
)

In [55]:
len(test.left_transfer_matrices)

1

In [56]:
len(left_gradients)

1

In [57]:
left_gradients[0]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [58]:
left_gradients[0].to_ndarray()

array([[ 6.82573586e-01+0.j, -4.53610246e-12+0.j],
       [-4.53609937e-12+0.j, -6.82573586e-01+0.j]])

In [59]:
left_gradients[0].shape

(2, 2)

In [60]:
gradient = (
    test.right_expectation
    *test.symmetry_transfer_matrix_singular_vals[0]
    *left_gradients[0].to_ndarray()
)
delta_U = np_Z

In [61]:
np.sum(gradient*delta_U)

(0.9306162804445224+0j)

In [62]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z + np_Z,],
    right_boundary_unitaries = [np_X, np_Z]
)

In [63]:
test2.compute_expectation()

array(1.86123256)

In [64]:
test2.expectation - test.expectation

0.9306162804445213

All good

## gradients_outside_right_boundary_unitaries

In [65]:
outside_gradients = gradients_outside_right_boundary_unitaries(
    psi,
    (
        test.left_expectation
        * test.symmetry_transfer_matrix_singular_vals[0]
        * test.right_transfer_vectors[-1]
    ),
    3,
    test.right_symmetry_index + 2
)

In [66]:
outside_gradients

[<npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(2, 2) labels=['p*', 'p']>]

In [67]:
outside_gradients[0].to_ndarray()

array([[0.46530814+0.j, 0.12035289+0.j],
       [0.12035289+0.j, 0.46530814+0.j]])

In [68]:
np.sum(outside_gradients[0].to_ndarray()*np_X)

(0.2407057831122734+0j)

In [112]:
test3 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z, np_I + np_X]
)

In [113]:
test.compute_expectation()

array(0.93061628)

In [114]:
test3.compute_expectation()

array(1.17132206)

In [115]:
test3.expectation - test.expectation

0.24070578311227275

In [116]:
np.sum(outside_gradients[1].to_ndarray()*np_X)

(0.23265407011115596+0j)

In [120]:
test4 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z, np_I, np_I + np_X]
)

In [121]:
test4.compute_expectation()

array(1.16327035)

In [122]:
test4.expectation - test.expectation

0.23265407011115102

## gradients_outside_left_boundary_unitaries

In [123]:
outside_gradients = gradients_outside_left_boundary_unitaries(
    psi,
    (
        test.right_expectation
        * test.symmetry_transfer_matrix_singular_vals[0]
        * test.left_transfer_vectors[-1]
    ),
    3,
    test.left_symmetry_index - 1
)

In [124]:
outside_gradients

[<npc.Array shape=(2, 2) labels=['p', 'p*']>,
 <npc.Array shape=(2, 2) labels=['p', 'p*']>,
 <npc.Array shape=(2, 2) labels=['p', 'p*']>]

In [125]:
outside_gradients[0].to_ndarray()

array([[0.46530812+0.j, 0.12035289+0.j],
       [0.12035289+0.j, 0.46530816+0.j]])

In [126]:
np.sum(outside_gradients[0].to_ndarray()*np_X)

(0.24070578311226953+0j)

In [127]:
test3 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z, np_I + np_X],
    right_boundary_unitaries = [np_X, np_Z]
)

In [128]:
test3.compute_expectation()

array(1.17132206)

In [129]:
test3.expectation - test.expectation

0.2407057831122681

In [134]:
outside_gradients[1].to_ndarray()

array([[0.46530814+0.j, 0.11632704+0.j],
       [0.11632704+0.j, 0.46530814+0.j]])

There's an error here, ignore for now.

In [130]:
np.sum(outside_gradients[1].to_ndarray()*np_X)

(0.23265407011115596+0j)

In [131]:
test4 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z, np_I, np_I + np_X],
    right_boundary_unitaries = [np_X, np_Z, np_I, np_X]
)

In [132]:
test4.compute_expectation()

array(0.29081759)

In [133]:
test4.expectation - test.expectation

-0.6397986928055761

## quadratic_expectation_gradient

In [188]:
test = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [1j*np_Z,],
    right_boundary_unitaries = [1j*np_X, 1j*np_Z]
)

In [189]:
test.compute_expectation()

array(0.-0.93061628j)

In [190]:
squared_expectation = np.abs(test.expectation)**2

In [191]:
squared_expectation

0.8660466614283958

In [192]:
test.compute_svd_approximate_expectation()

(-0-0.9306162804445224j)

In [193]:
right_gradients = expectation_gradients(
    psi,
    test.right_transfer_matrices,
    test.right_symmetry_index+1,
    left_environment=test.right_projected_symmetry_state,
    right_environment=None
)

In [194]:
len(test.right_transfer_matrices)

2

In [195]:
len(right_gradients)

2

In [196]:
gradient = (
    test.left_expectation
    *test.symmetry_transfer_matrix_singular_vals[0]
    *right_gradients[0].to_ndarray()
)

In [197]:
quad_gradient = quadratic_expectation_gradient(gradient, test.expectation)

In [198]:
gradient

array([[-0.12035289+0.j, -0.46530814+0.j],
       [-0.46530814+0.j, -0.12035289+0.j]])

In [199]:
quad_gradient

array([[-0.+0.22400472j, -0.+0.86604666j],
       [-0.+0.86604666j, -0.+0.22400472j]])

Try different perturbations

In [212]:
delta_U = 1e-2*1j*np_X

In [213]:
delta_U

array([[0.+0.j  , 0.+0.01j],
       [0.+0.01j, 0.+0.j  ]])

In [214]:
test.expectation

array(0.-0.93061628j)

In [215]:
np.sum(delta_U*gradient)

-0.009306162804445222j

In [216]:
first_order_correction_from_gradient_one_site(quad_gradient, delta_U)

0.017320933228567932

In [217]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [1j*np_Z,],
    right_boundary_unitaries = [1j*np_X + delta_U, 1j*np_Z]
)

In [218]:
test2.compute_expectation()

array(0.-0.93992244j)

In [219]:
test2.expectation - test.expectation

-0.009306162804445184j

In [220]:
np.abs(test2.expectation)**2 - squared_expectation

0.017407537894710723

In [230]:
delta_U = 1e-2*1j*np_I

In [231]:
delta_U

array([[0.+0.01j, 0.+0.j  ],
       [0.+0.j  , 0.+0.01j]])

In [232]:
test.expectation

array(0.-0.93061628j)

In [233]:
np.sum(delta_U*gradient)

-0.00240705783112258j

In [234]:
first_order_correction_from_gradient_one_site(quad_gradient, delta_U)

0.004480094411228304

In [235]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [1j*np_Z,],
    right_boundary_unitaries = [1j*np_X + delta_U, 1j*np_Z]
)

In [236]:
test2.compute_expectation()

array(0.-0.93302334j)

In [237]:
test2.expectation - test.expectation

-0.0024070578311222723j

In [238]:
np.abs(test2.expectation)**2 - squared_expectation

0.004485888338630173

## linear_expectation_gradient

In [239]:
abs_expectation = np.abs(test.expectation)

In [240]:
abs_expectation

0.9306162804445213

In [241]:
lin_gradient = linear_expectation_gradient(gradient, test.expectation)

In [242]:
gradient

array([[-0.12035289+0.j, -0.46530814+0.j],
       [-0.46530814+0.j, -0.12035289+0.j]])

In [243]:
lin_gradient

array([[-0.+0.12035289j, -0.+0.46530814j],
       [-0.+0.46530814j, -0.+0.12035289j]])

Try different perturbations

In [244]:
delta_U = 1e-2*1j*np_X

In [245]:
delta_U

array([[0.+0.j  , 0.+0.01j],
       [0.+0.01j, 0.+0.j  ]])

In [246]:
test.expectation

array(0.-0.93061628j)

In [247]:
first_order_correction_from_gradient_one_site(lin_gradient, delta_U)

0.009306162804445222

In [248]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [1j*np_Z,],
    right_boundary_unitaries = [1j*np_X + delta_U, 1j*np_Z]
)

In [249]:
test2.compute_expectation()

array(0.-0.93992244j)

In [250]:
test2.expectation - test.expectation

-0.009306162804445184j

In [251]:
np.abs(test2.expectation) - abs_expectation

0.009306162804445184

In [252]:
delta_U = 1e-2*1j*np_I

In [253]:
delta_U

array([[0.+0.01j, 0.+0.j  ],
       [0.+0.j  , 0.+0.01j]])

In [254]:
first_order_correction_from_gradient_one_site(lin_gradient, delta_U)

0.00240705783112258

In [255]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [1j*np_Z,],
    right_boundary_unitaries = [1j*np_X + delta_U, 1j*np_Z]
)

In [256]:
test2.compute_expectation()

array(0.-0.93302334j)

In [257]:
test2.expectation - test.expectation

-0.0024070578311222723j

In [258]:
np.abs(test2.expectation) - abs_expectation

0.0024070578311222723

Good enough!

# Old code

In [85]:
x = 7

In [86]:
isinstance(x, int)

True

In [87]:
isinstance('a', int)

False

In [88]:
help(range)

Help on class range in module builtins:

class range(object)
 |  range(stop) -> range object
 |  range(start, stop[, step]) -> range object
 |  
 |  Return an object that produces a sequence of integers from start (inclusive)
 |  to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
 |  start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
 |  These are exactly the valid indices for a list of 4 elements.
 |  When step is given, it specifies the increment (or decrement).
 |  
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash

In [89]:
kd_00 = np.array([[1,0], [0, 0]])
kd_01 = np.array([[0,1], [0, 0]])
kd_10 = np.array([[0,0], [1, 0]])
kd_11 = np.array([[0,0], [0, 1]])

In [90]:
kd_arrays = [kd_00, kd_01, kd_10, kd_11]

In [91]:
kd_tests = [
    SymmetryActionWithBoundaryUnitaries(
        psi,
        [np_X, np_I]*50,
        left_boundary_unitaries = [np_Z,],
        right_boundary_unitaries = [m, np_Z]
    )
    for m in kd_arrays
]

In [92]:
for t in kd_tests:
    t.compute_expectation()

In [93]:
kd_expectations = [t.expectation for t in kd_tests]

In [94]:
kd_expectations

[array(0.12035289), array(0.46530814), array(0.46530814), array(0.12035289)]

In [95]:
gradient

array([[ 4.65308140e-01+0.j, -3.09224594e-12+0.j],
       [-3.09224384e-12+0.j, -4.65308140e-01+0.j]])

In [96]:
0.17654904/0.12035289

1.466928131098472

In [97]:
0.68257359/0.46530814

1.4669281091880317

In [98]:
(0.68257359/0.46530814)**(-1)

0.6816966651170902

In [99]:
test.symmetry_transfer_matrix_singular_vals

array([4.99357640e-01, 5.64338811e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
      

In [100]:
test.left_expectation

(1.3651471712007195+0j)

In [101]:
test.left_expectation*(test.symmetry_transfer_matrix_singular_vals[0])

(0.6816966698367003+0j)